In [ ]:
#Grundlage: https://pages.databricks.com/rs/094-YMS-629/images/Blog_%20A%20Multivariate%20Time%20Series%20Forecasting%20Appliance%20Energy%20Usage.html?_ga=2.215927575.148871660.1597125934-849368782.1596610145

In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [19]:
###read in data
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))
os.chdir("C:/Users/S.Stroka/Desktop/Data")

df_0 = pd.read_excel("Product_Group_FOL_EOL_short.xlsx")
df_0 = pd.DataFrame(df_0)

df_1 = pd.read_excel("Product_Group_FF.xlsx")
df_1 = pd.DataFrame(df_1)

Files in 'C:\\Users\\S.Stroka\\Desktop\\Data': ['Product_Group_FF.xlsx', 'Product_Group_FOL_EOL_short.xlsx']


In [20]:
df_1.rename(columns = {'Calendar Day':'Date'}, inplace = True)
FF_Plan = df_1[['Date','Product Group','FF Plan']]
df_1 = df_1.iloc[:,1:4]

df_0.rename(columns = {'Calendar Day':'Date'}, inplace = True)
df_0.head()
df_0 = df_0.fillna(0)
df_0.head()

#Overall Link
df = pd.merge(df_0,df_1, how = 'outer', on = ['Date','Product Group'])
df.head()

,Date,Product Group,WIP AVG (EOL),MOVES (EOL),WIP AVG (FOL),MOVES (FOL),FF
0,2019-07-26,INGAALP_2000,8.990,50.0,15.632,98.0,NaN
1,2019-07-26,INGAALP_LC,121.420,525.0,47.499,384.0,3.869375
2,2019-07-26,INGAALP_NS,176.366,465.0,112.780,565.0,3.913862
3,2019-07-26,INGAALP_STD,249.565,1046.0,153.220,631.0,4.569528
4,2019-07-26,PP_TF3_13,0.000,0.0,24.000,24.0,NaN


In [195]:
prod_groups = np.unique(df[['Product Group']])
#prod_groups
prod_groups[0]
#len(prod_groups)

'HPL_BAR'

In [197]:
i = 3 #i-te Product group (distinct)
df_single = df[df["Product Group"]==prod_groups[i]]

In [199]:
df_single.iloc[:,6:7]
df_single.head()
#df_single.size

,Date,Product Group,WIP AVG (EOL),MOVES (EOL),WIP AVG (FOL),MOVES (FOL),FF
1,2019-07-26,INGAALP_LC,121.420,525.0,47.499,384.0,3.869375
35,2019-07-27,INGAALP_LC,119.703,684.0,48.604,145.0,NaN
69,2019-07-28,INGAALP_LC,141.578,272.0,27.060,48.0,NaN
103,2019-07-29,INGAALP_LC,136.498,162.0,25.000,50.0,7.615311
137,2019-07-30,INGAALP_LC,139.889,392.0,2.701,50.0,4.083300


In [200]:
###Data clustering in time series arrays
x=np.asarray(df_single[['WIP AVG (EOL)', 'MOVES (EOL)', 'WIP AVG (FOL)', 'MOVES (FOL)']]) #Date not used as explaining variable
y=np.asarray(df_single['FF'])

##not necessary, already in right format
#x = np.random.rand(x.shape[0],x.shape[1])
#y = np.random.rand(y.shape[0],)

In [116]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x,y, test_size = 0.2, random_state = 196, shuffle = False)
temp = (train_x.shape,train_y.shape,test_x.shape,test_y.shape)
temp

((286, 4), (286,), (72, 4), (72,))

In [201]:
#entweder timeseriesGenerator(keras) oder selbst konverten
#n_input also batchsize: The batch size determines the number of samples before a gradient update takes place.
n_input = 2 #overlapping windows of length = 4 different features //timesteps
train_generator = TimeseriesGenerator(train_x,train_y,length=n_input, sampling_rate=1, batch_size=1)
test_generator = TimeseriesGenerator(test_x,test_y,length=n_input, sampling_rate=1, batch_size=1) #batch_size = timesteps (later)

In [202]:
# number of samples
print('Samples: %d' % len(generator_x))


Samples: 356


In [124]:
# print each sample
for i in range(len(generator_x)):
	x, y = generator_x[i]
	print('%s => %s' % (x, y))

[[[121.42  525.     47.499 384.   ]
  [119.703 684.     48.604 145.   ]]] => [[141.578 272.     27.06   48.   ]]
[[[119.703 684.     48.604 145.   ]
  [141.578 272.     27.06   48.   ]]] => [[136.498 162.     25.     50.   ]]
[[[141.578 272.     27.06   48.   ]
  [136.498 162.     25.     50.   ]]] => [[139.889 392.      2.701  50.   ]]
[[[136.498 162.     25.     50.   ]
  [139.889 392.      2.701  50.   ]]] => [[ 84.804 312.      0.      0.   ]]
[[[139.889 392.      2.701  50.   ]
  [ 84.804 312.      0.      0.   ]]] => [[ 67.011 351.     13.565  72.   ]]
[[[ 84.804 312.      0.      0.   ]
  [ 67.011 351.     13.565  72.   ]]] => [[ 57.28  206.     48.513 387.   ]]
[[[ 67.011 351.     13.565  72.   ]
  [ 57.28  206.     48.513 387.   ]]] => [[ 68.987 180.     33.079 228.   ]]
[[[ 57.28  206.     48.513 387.   ]
  [ 68.987 180.     33.079 228.   ]]] => [[65.717 78.    24.     0.   ]]
[[[ 68.987 180.     33.079 228.   ]
  [ 65.717  78.     24.      0.   ]]] => [[ 67.184 228.     65.4

In [203]:
train_X, train_Y = train_generator[0]
test_X, test_Y = test_generator[0]

train_samples = train_X.shape[0]*len(train_generator)
test_samples = test_X.shape[0]*len(test_generator)

print("Total Records (n): {}".format(df.count()))
#print("Total Records after adjusting for 24 hours: {}".format(len(x)))
print("Number of samples in training set (.8 * n): train_x = {}".format(train_x.shape[0]))
print("Number of samples in testing set (.2 * n): test_x = {}".format(test_x.shape[0]))
print("Size of individual batches: {}".format(test_X.shape[1]))
print("Number of total samples in training feature set: {}".format(train_samples))
print("Number of samples in testing feature set: {}".format(test_samples))

Total Records (n): Date             11831
Product Group    11831
WIP AVG (EOL)    11823
MOVES (EOL)      11823
WIP AVG (FOL)    11823
MOVES (FOL)      11823
FF                3976
dtype: int64
Number of samples in training set (.8 * n): train_x = 286
Number of samples in testing set (.2 * n): test_x = 72
Size of individual batches: 2
Number of total samples in training feature set: 284
Number of samples in testing feature set: 70


In [187]:
from time import time
!pip install dbutils
from keras.callbacks import TensorBoard
from tensorboard import notebook

tb_dir = '/tmp/tensorflow_log_dir/{}'.format(time())
tensorboard = TensorBoard(log_dir = tb_dir)

import dbutils
dbutils.tensorboard.start(tb_dir)

In [191]:
###Model Training
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping
#!pip install mlflow
import mlflow
import mlflow.keras

In [204]:
units = 128
num_epoch = 5000
learning_rate = 0.00144

with mlflow.start_run(experiment_id=3133492, nested=True):

  model = Sequential()
  model.add(CuDNNLSTM(units, input_shape=(train_X.shape[1], train_X.shape[2])))
  model.add(LeakyReLU(alpha=0.5)) 
  model.add(Dropout(0.1))
  model.add(Dense(1))

  adam = Adam(lr=learning_rate)
  # Stop training when a monitored quantity has stopped improving.
  callback = [EarlyStopping(monitor="loss", min_delta = 0.00001, patience = 50, mode = 'auto', restore_best_weights=True), tensorboard] 

  # Using regression loss function 'Mean Standard Error' and validation metric 'Mean Absolute Error'
  model.compile(loss='mse', optimizer=adam, metrics=['mae'])

# fit network
  history = model.fit_generator(train_generator, \
                                epochs=num_epoch, \
                                validation_data=test_generator, \
                                callbacks = callback, \
                                verbose=2, \
                                shuffle=False, \
                                initial_epoch=0)

  mlflow.log_param("Units", units)
  mlflow.log_param("Epochs", num_epoch)
  mlflow.log_param("Learning Rate", learning_rate)
  mlflow.log_param("Lags cosidered", timesteps)

#   Return loss value and metric value
  score = model.evaluate_generator(test_generator, verbose=0)   
  mlflow.log_metric("Test Loss", score[0]) 
  mlflow.log_metric("MAE", score[1])   
  mlflow.log_metric("Actual Epochs", len(history.history['loss']))
  mlflow.keras.log_model(model, "LSTM Model")
    
  # The model can be saved for future use and move to production
#   mlflow.keras.save_model(model1, "/dbfs/ved-demo/timeseries/best-appliance-model/")

MlflowException: Could not find experiment with ID 3133492

In [160]:
conda install 

NameError: name 'conda_env' is not defined

In [ ]:
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator_x, steps_per_epoch=1, epochs=500, verbose=0)
# make a one step prediction out of sample
x_input = array([[90, 95], [100, 105]]).reshape((1, n_input, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

In [23]:
df.dtypes

Calendar Day     datetime64[ns]
Product Group            object
WIP AVG (EOL)           float64
MOVES (EOL)             float64
WIP AVG (FOL)           float64
MOVES (FOL)             float64
dtype: object